In [72]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import unicodedata

Team name constants

In [7]:
team_tags_dico = {
    "scl_tags" : ["scl","scald"],
    "iwg_tags" : ["iwg"]
}

In [ ]:
driver_options = webdriver.FirefoxOptions()
driver_options.add_argument("--headless")
driver = webdriver.Firefox(options=driver_options)

In [23]:
def getBlue_picks_bans(soup,name):
    picks = soup.find(attrs={"class": "roomPickColumn blue"}).get_text("|")
    picks = picks.split("|")
    bans = [x['alt'] for x in soup.find(attrs={"class": "roomBanRow blue"}).find_all('img')]
    return picks,bans, name

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html,"html.parser")
getBlue_picks_bans(soup,"blue")

In [25]:
def getRed_picks_bans(soup,name):
    picks = soup.find(attrs={"class": "roomPickColumn red"}).get_text("|")
    picks = picks.split("|")
    bans = [x['alt'] for x in soup.find(attrs={"class": "roomBanRow red"}).find_all('img')]
    return picks, bans, name

In [ ]:
getRed_picks_bans(soup,"red")

In [70]:
def get_side_by_tag(soup,team_reference_dict):
    #TODO ATTENTION, il faut récupérer le nom de l'équipe, et si dans le nom qui contient Scald il y a des chiffres, alors
    # récupérer chiffre et split pour obtenir date de la game et numéro de la game
    blue_team = unicodedata.normalize("NFKD",soup.find(attrs={"class" : "roomReadyBackground roomReadyBackgroundblue"}).previous_sibling.get_text())
    red_team = unicodedata.normalize("NFKD",soup.find(attrs={"class" : "roomReadyBackground roomReadyBackgroundred"}).previous_sibling.get_text())
    return blue_team, red_team, team_reference_dict

In [ ]:

get_side_by_tag(soup,"test")

In [4]:
def scraping_draft(draft_url,team_reference_dict):

    driver.get(draft_url)
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    blue_name , red_name = get_side_by_tag(soup,team_reference_dict)

    blue = getBlue_picks_bans(soup,blue_name)
    red = getRed_picks_bans(soup,red_name)
    
    
    return blue, red